In [ ]:
import pandas as pd
import urllib.request
import re
import matplotlib.pyplot as plt
import datetime
import win32com.client

%matplotlib inline

# NATE SILVER

In [ ]:
c = pd.read_csv("https://projects.fivethirtyeight.com/2020-general-data/presidential_scenario_analysis_2020.csv")
filtered_scenarios = c[(c.scenario_description == "Biden wins popular vote") | (c.scenario_description == "Trump wins popular vote")]
df = filtered_scenarios[['scenario_description', 'probability', 'modeldate']]
df_nate = df.pivot(index = 'modeldate', columns = 'scenario_description', values='probability')
df_nate.columns = ['biden', 'trump']
df_nate.index = pd.to_datetime(df_nate.index)

In [ ]:
df_nate.plot(title='NateSilver')

# GJP

In [ ]:
# Get HTML from website
url = "https://goodjudgment.io/superforecasts/"
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36',} 

request=urllib.request.Request(url,None,headers) #The assembled request
response = urllib.request.urlopen(request)
data = response.read()

# Parse data from website
chart_gen = re.findall("function tdrawChart1338.+?options1338", str(data))[0]
data_points = re.findall("(?<=\[)\".+?(?=\])", chart_gen)

# Clean data
df_gjp = pd.DataFrame([sub.split(",") for sub in data_points], columns=['date', 'biden', 'trump', 'others'])
df_gjp['date'] = pd.to_datetime(df_gjp['date'].str.replace("\"", "") + " 2020")
df_gjp.set_index('date', inplace=True)
df_gjp = df_gjp[['biden', 'trump', 'others']].astype('int32')

In [ ]:
df_gjp.plot()

# Betting odds from Macrobond

In [ ]:
# Can only be run on Macrobond PC

def _convert(tuple_time):
    return [_convert_single(x) for x in tuple_time]

def _convert_single(time):
    return pd.Timestamp(time.timestamp(), unit = 's')

def mb_to_pd(mb_obj, label='values'):
    return pd.DataFrame({'dates':_convert(mb_obj.DatesAtStartOfPeriod), label: mb_obj.Values}).set_index('dates')

c = win32com.client.Dispatch("Macrobond.Connection")
d = c.Database
trump, biden = d.FetchSeries(["pit_poli_3698_7943", "pit_poli_3698_7940"])
df_bet = mb_to_pd(trump, label="trump").join(mb_to_pd(biden, label="biden"))

In [ ]:
df_bet.plot()

# Consolidated

In [ ]:
ax = pd.DataFrame({'betting odds':df_bet['biden'], 'nate silver':df_nate['biden'], 'gjp':df_gjp['biden']/100}).plot(title='Prob of Biden Win')
ax.set_xlim(datetime.date(2020, 1, 1))